# このPythonプログラムで何を実現するのか
- サンプル書誌情報一覧を作成する
- 和書の書誌情報はCiNii Booksから、洋書の書誌情報はGoogle Booksから取得する
- 国際標準図書番号ISBN(International Standard Book Number)を使って、書籍の書誌情報データにアクセスし、必要な情報のみを抽出する
- CiNiiおよびGoogle Booksは、APIという機能を提供しており、このAPIにISBNのデータを渡すと、書誌情報をコンピュータプログラミングで処理しやすい形式のデータとして出力してくれる

# この演習で何を学べるのか
- 標準（※誰でもいつでもどこでも同じように使えるという意味）的に作られたデータの利点
- 標準データとしてよく用いられるXMLとJSONデータの基本構造
- Webスクレイピング（Webから機械的に情報を取得する作業）の基本
- Pythonの基礎（基本的なデータ型（辞書・リスト・JSON）、関数の定義、条件分岐、ファイルの読み書き、コメントの挿入など）


# 下準備
- 左上のファイルから、ファイルのコピーを保存する（あるいはノートブック（「20231220デジタル文献情報学研究法バーコード演習.ipynb」）をアップロードする）（注参照）
- Googleドライブ上のファイルにアクセスできるようにドライブをマウントする
- ISBNを格納するCSVファイルを、バーコードリーダーを使って別途作成する
- ↑本のバーコードを読み込むとISBNが取得できるので、その機能を使う

- 注：新潟大学のアカウントでは、セキュリティの問題でノートブックを開けないように設定されているようです。この実習では、個人のGoogleアカウントを使用するようにしてください。アカウントの切り替えだけでは開けないため、一度個人のGoogleアカウントでColaboratoryにアクセスし、ノートブックをアップロードします。

### Googleドライブ上のファイルにアクセスできるようにドライブをマウントする
以下のスクリプトを実行し、"Mounted at /content/drive"と表示されれば完了

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 【補足】マウントとは

コンピュータのストレージデバイスやファイルシステムを、システムのディレクトリ構造に組み込むことを指します。
ここでのマウントを例にもう少しわかりやすく説明すると、以下のようになります。
1. Google Colabで共有しているこのPythonファイル（.ipynb）は、誰でも見ることができるし、誰でも動かすことができます。ただし、ここに書かれたコードを実行するためには、ファイル（isbn.csv）を読み込む必要があります。なぜなら、共有されているこのPythonファイルにisbn.csvというファイルは含まれていないからです。
2. isbn.csvというファイルを読み込むためには、自身のGoogleドライブのデータとこのPythonコードを接続しなければなりません。PythonのコードだけでGoogleドライブにアクセスし、操作できるようにするための作業が「マウント」です。
3. Googleドライブをマウントしたら、ここに書かれたコードを使ってドライブ内のファイルを開いたり、編集したり、保存したりできるようになります。


### マウントできたら、isbn.csvをアップロードする
左に表示されているフォルダアイコンからdrive→MyDrive→Colab Notebooksにアクセスし、右の三点を押して作成したファイルをアップロードする

In [ ]:
# 必要なモジュール・ライブラリのインポート（読み込み）
import requests
import bs4
import os
import time
import pandas
import json
import csv

### 【補足】モジュール・ライブラリとは

Pythonの「モジュール」とは、関連する関数、クラス、変数などをまとめたファイルのことです。モジュールはPythonのコードを再利用可能な部品として整理するために使用されます。これにより、コードをより管理しやすく、読みやすく、再利用しやすくなります。

1. requests
- 用途: ウェブページやAPIからデータを取得するために使用されるHTTPライブラリです。
- 特徴: GET、POSTなどのHTTPリクエストを簡単に送信でき、レスポンスの処理も容易に行えます。
2. bs4 (Beautiful Soup)
- 用途: HTMLやXMLファイルからデータを抽出するためのライブラリです。
- 特徴: タグに基づいてウェブページの解析が行え、特定の要素や属性を簡単に抽出できます。
3. os
- 用途: オペレーティングシステムの機能とのやり取りを行うための標準ライブラリです。
- 特徴: ファイルやディレクトリの操作、パスの取得、環境変数の操作などが可能です。
4. time
- 用途: 日付や時刻に関連する操作を行うための標準ライブラリです。
- 特徴: 現在の時刻の取得、プログラムの実行を一時停止するためのスリープ機能、タイムスタンプの取得などができます。
5. pandas
- 用途: データ分析やデータ操作に特化した強力なライブラリです。
- 特徴: データフレームとシリーズというデータ構造を用いて、データの読み込み、加工、集計などが行えます。
6. json
- 用途: JSONデータの解析や生成を行うための標準ライブラリです。
- 特徴: PythonオブジェクトとJSONデータの相互変換（シリアライズ、デシリアライズ）が可能です。
7. csv
- 用途: CSVファイルの読み書きを行うための標準ライブラリです。
- 特徴: CSVデータの読み込みや書き出しを簡単に行え、データの処理や解析に利用できます。

# 後の処理で必要になる関数を先に定義しておく

In [ ]:
def make_dict(author, title, publisher, year, isbn):
  # make_dictは任意で作成できる関数の名前。author以下は、この関数内でのみ有効な「ローカル変数」として機能する

    d = {} # dという名前の空の辞書を用意しておく（辞書はkeyとvalueからなる要素の集合）
    d['著者'] = author # dという辞書に、key='著者' value='author'という値を入れている
    d['題名'] = title
    d['出版社'] = publisher
    d['出版年'] = year
    d['ISBN'] = isbn

    return d
    # この関数によって、どのような値が返されるかはこのreturnによって決まる
    # d = {'著者': author, '題名': title, '出版社': publisher, '出版年': year, 'ISBN': isbn} となるはず

# 関数を定義しただけでは何も起こらない。実際に動かすには、関数を呼び出し、引数（ひきすう、argument）を指定することが必要

### 【補足】変数とは

変数とは、プログラミングにおいてデータを一時的に保存するための「容器」のようなものです。変数には名前があり、この名前を使って保存したデータにアクセスしたり、データを変更したりできます。

変数を使うことで、プログラム内で数値やテキスト、その他のデータを簡単に扱うことができます。例えば、ある数値を何度も使いたい場合、その数値を変数に保存しておけば、その後のプログラムの中でその変数名を使ってその数値を参照できます。

#### ローカル変数とグローバル変数

ローカル変数は、関数やメソッド内で定義され、その特定の関数やメソッド内でのみアクセス可能な変数です。
- ローカル変数は、その関数やメソッドが呼び出されたときに作成され、関数の実行が終了すると消滅します。
- 同じ名前のローカル変数が別の関数内で定義されていても、それらは互いに影響しません。

グローバル変数は、プログラム全体でアクセス可能な変数で、通常は関数の外部で定義されます。
- グローバル変数はプログラムのどこからでもアクセスできます。
- グローバル変数はプログラムが終了するまで存在し続けます。
- グローバル変数を多用すると、プログラムが複雑になり、バグの原因になることがあります。

### 【補足】関数とは

関数とは、ある特定の仕事をするためのコードのかたまりで、必要に応じて何度も再利用できるように設計されています。プログラミングにおける関数は、特定の「入力」（パラメータや引数と呼ばれます）を受け取り、「出力」（結果）を返すことができます。
cf. モジュール、ライブラリ

例えば、Excelで合計を求めるときには、=SUM(A1:A8)のように記述することで簡単に答えを導き出すことができます。このSUMは「SUM関数」のこと。つまり、いくつかの値（Excelではセルの範囲）を引数として受け取ると、合計の値を返す関数です。

In [ ]:
# SUM関数の中身をPython風に記述するとこうなる
def my_sum(iterable, start=0):
    total = start
    for element in iterable:
        total += element
    return total

# 例として、リスト [1, 2, 3, 4, 5] の合計を計算する
result = my_sum([1, 2, 3, 4, 5])
print(result)  # 出力は 15

### 【補足】引数とは

引数（パラメータとも呼ばれる）は、プログラミングにおいて関数やメソッドに渡される値やデータのことです。引数を使うことで、関数に外部から情報を提供し、その関数の動作をカスタマイズしたり、特定のタスクを実行させたりすることができます。

引数の役割は以下のようなものです。
- データの受け渡し: 引数を通じて、関数に必要なデータを渡すことができます。
- 柔軟性の提供: 同じ関数を異なる引数で複数回呼び出すことで、異なる結果を得ることができます。

### 【補足】リスト型と辞書型

1. リスト（[ ]で囲まれた中にカンマでデータが並列された）型のデータ
- 以下のようなリストの例があったとする。バナナという値だけを取りたい場合には、
  - sample_list = ["リンゴ", "みかん", "バナナ"]
  - sample_list[2] というように、最初から2番目（インデックスは0から始まる）の値を指定すると、バナナが得られる
- 同様に、リンゴを得るにはどうしたらいいか？
2.  辞書（{ }で囲まれた中に、: の前後に見出し語（キー）と意味（値もしくはバリュー）がセットで格納された）型のデータ
- 以下のような辞書の例があったとする。年齢のデータが欲しいとすると、
  - sample_dict = {"学籍番号": "123456", "氏名": "千葉太郎", "年齢": "21", "得意科目": ["統計", "文学", "英語"]}
  - sample_dict["年齢"] というように、変数名の後に["キーの名前"] を続けて入力すれば、そのキーに対応する値を得られる

In [ ]:
sample_list = ["リンゴ", "みかん", "バナナ"]
sample_list[len(sample_list)-1]

'バナナ'

In [ ]:
sample_dict = {"学籍番号": "123456", "氏名": "千葉太郎", "年齢": "21", "得意科目": ["統計", "文学", "英語"]}
sample_dict["得意科目"][0]

'統計'

# CiNiiのAPIから返ってくるXMLデータの例を確認してみよう

### 【補足】APIとは

API（Application Programming Interface）とは「アプリケーションプログラミングインターフェース」の略で、ソフトウェアやアプリケーションが他のソフトウェアやサービスと情報をやり取りするための一連の規則や仕様です。簡単に言うと、APIは異なるソフトウェア間で「会話」をするための方法を提供します。

Webスクレイピング（ウェブページから情報を自動的に収集し、抽出するプロセス）の際には、はじめに相手方のサイトがAPIを提供しているか確認します。APIが提供されていれば、API経由で情報を取得するようにしましょう。

APIを使う利点はたくさんあります。
1. 効率性: APIを使用すると、既に他の開発者や企業によって構築された機能にアクセスできます。これにより、ゼロから機能を開発する手間と時間を節約できます。
2. 標準化と一貫性: APIは一般的に標準化された方法でデータや機能を提供します。これにより、異なるシステム間での互換性と一貫性が保たれます。
3. 安全性: APIは、データや機能へのアクセスを制御するためのセキュリティ層を提供します。これにより、直接データベースや内部システムにアクセスするよりも安全にデータを扱うことができます。
4. メンテナンスとアップデートの容易さ: APIを通じてサービスを利用する場合、APIの提供者がバックエンドのメンテナンスやアップデートを行います。これにより、エンドユーザーは常に最新の機能やデータにアクセスできます。
5. スケーラビリティ: APIは通常、大量のリクエストやデータを処理するように設計されています。これにより、アプリケーションのスケールアップが容易になります。
6. コミュニティとサポート: 人気のあるAPIは大きな開発者コミュニティを持っていることが多く、ドキュメント、チュートリアル、フォーラムなどの豊富なリソースが提供されます。

まとめると、APIを使用することで、開発プロセスが高速化し、より安全で、メンテナンスが容易になり、最終的にはより優れた製品やサービスを提供することができます。

例えば、CiNii Booksの以下の書籍。
ソースコードはこちら→"view-source:https://ci.nii.ac.jp/ncid/BB23233235"
APIは下に載せた通り。
どちらが情報を取りやすいでしょうか？

1.   \<feed>というタグを最上位とする階層構造
2.   \<entry>というタグの下層に、必要そうな書誌情報がある
3.   今回は、著者名・題名・出版社・出版年・ISBNのデータが欲しい
4.   どのタグの情報を取ればいいだろう？

```
<feed xml:lang="ja">
  <title>CiNii Books OpenSearch - 978-4-534-05472-2</title>
  <link href="https://ci.nii.ac.jp/books/search/?advanced=true&isbn=978-4-534-05472-2&count=20&sortorder=1"/>
  <link rel="self" type="application/atom+xml" href="https://ci.nii.ac.jp/books/opensearch/search?isbn=978-4-534-05472-2"/>
  <id>https://ci.nii.ac.jp/books/opensearch/search?isbn=978-4-534-05472-2
  </id>
  <updated>2022-09-20T22:17:31+09:00</updated>
  <opensearch:totalResults>1</opensearch:totalResults>
  <opensearch:startIndex>0</opensearch:startIndex>
  <opensearch:itemsPerPage>1</opensearch:itemsPerPage>
  <entry>
    <title>Google流資料作成術</title>
    <link href="https://ci.nii.ac.jp/ncid/BB23233235"/>
    <link rel="alternate" type="application/rdf+xml" href="https://ci.nii.ac.jp/ncid/BB23233235.rdf"/>
    <id>https://ci.nii.ac.jp/ncid/BB23233235</id>
    <author>
      <name>コール・ヌッスバウマー・ナフリック著 ; 村井瑞枝訳</name>
    </author>
    <dc:publisher>日本実業出版社</dc:publisher>
    <prism:publicationDate>2017</prism:publicationDate>
    <updated>2017</updated>
    <dcterms:hasPart>urn:isbn:9784534054722</dcterms:hasPart>
    <cinii:ownerCount>134</cinii:ownerCount>
  </entry>
</feed>
```

# CiNiiのAPIから必要な情報を取得するための関数を定義

In [ ]:
def get_from_cinii(isbn, res_xml): # res_xmlという変数にはxmlデータが格納されている

    author = res_xml.entry.author.text.strip()
    # res_xmlの中の<entry>タグ、さらにその中の<author>タグにアクセス
    # .textでそのタグの中のテキストを取得
    # .strip()で文字列の前後にある余分な空白（スペースや改行など）を削除
    title = res_xml.entry.title.text.strip()
    publisher = res_xml.entry.select('publisher')[0].text.strip()
    # res_xmlの中の<entry>タグにアクセス
    # .select('publisher')で<entry>タグないの<publisher>タグを選択。取得されたデータはリスト型で返される
    # [0]は、.selectで作成されたリストの０番目（一番最初の要素）を取得している。<publisher>タグが複数存在する場合があるため
    year = res_xml.entry.select('publicationDate')[0].text.strip()
    # book_id = res_xml.entry.id.text.strip()
    # <entry><id>xxxx</id></entry>

    d = make_dict(author, title, publisher, year, isbn)
    # dという辞書を作成している。make_dictは、上で作成したmake_dictという関数を呼び出している
    # この１行だけで、make_dictとして定義された処理が一気に行われる
    # ここではreturn（辞書型）がdという変数に格納されているが、これはmake_dictで作成したdという辞書とは別のもの。
    # 関数内で定義したローカル変数は関数の外に引き継げないので、ここで新たに変数名を定義する必要がある
    d['情報源'] = 'cinii'
    # make_dictの呼び出しによって作成したdという辞書に、新たにkey='情報源' value='cinii'を追加

    return d
    # d = {'著者': author, '題名': title, '出版社': publisher, '出版年': year, 'ISBN': isbn, '情報源': 'cinii'} となるはず
    # 上の例だと、d = {'著者': 'コール・ヌッスバウマー・ナフリック著 ; 村井瑞枝訳', '題名': 'Google流資料作成術', '出版社': '日本実業出版社', '出版年': '2017', 'ISBN': 9784534054722, '情報源': 'cinii'} となるはず
    # isbnは元々csvファイルで作成しているため、ここで新たに取得する必要はない


# 次に、Google BooksのAPIから返ってくるJSONデータの例を確認してみよう


1.  今度は、XML形式ではないが、リストと辞書が混ざった階層構造になっている（JSON形式）
2.  著者名・題名・出版社・出版年・ISBNのデータを取りたいが、どのキーにアクセスして値を取得したらいいか？
  - その前に、上記sample_dictから応用問題。千葉太郎の得意科目のうち、文学という値を取るにはどうしたらいいか？



```
{
  "kind": "books#volumes",
  "totalItems": 1,
  "items": [
    {
      "kind": "books#volume",
      "id": "4Y18AQAAQBAJ",
      "etag": "2Y0fAiyrl8w",
      "selfLink": "https://www.googleapis.com/books/v1/volumes/4Y18AQAAQBAJ",
      "volumeInfo": {
        "title": "Writing History in the Digital Age",
        "authors": [
          "Jack Dougherty",
          "Kristen Nawrotzki"
        ],
        "publisher": "University of Michigan Press",
        "publishedDate": "2013-10-28",
        "description": "\"Writing History in the Digital Age began as a one-month experiment in October 2010, featuring chapter-length essays by a wide array of scholars with the goal of rethinking traditional practices of researching, writing, and publishing, and the broader implications of digital technology for the historical profession. The essays and discussion topics were posted on a WordPress platform with a special plug-in that allowed readers to add paragraph-level comments in the margins, transforming the work into socially networked texts. This first installment drew an enthusiastic audience, over 50 comments on the texts, and over 1,000 unique visitors to the site from across the globe, with many who stayed on the site for a significant period of time to read the work. To facilitate this new volume, Jack Dougherty and Kristen Nawrotzki designed a born-digital, open-access platform to capture reader comments on drafts and shape the book as it developed. Following a period of open peer review and discussion, the finished product now presents 20 essays from a wide array of notable scholars, each examining (and then breaking apart and reexamining) how digital and emergent technologies have changed the ways that historians think, teach, author, and publish\"--",
        "industryIdentifiers": [
          {
            "type": "ISBN_13",
            "identifier": "9780472052066"
          },
          {
            "type": "ISBN_10",
            "identifier": "0472052063"
          }
        ],
        "readingModes": {
          "text": false,
          "image": true
        },
        "pageCount": 283,
        "printType": "BOOK",
        "categories": [
          "History"
        ],
        "maturityRating": "NOT_MATURE",
        "allowAnonLogging": false,
        "contentVersion": "preview-1.0.0",
        "panelizationSummary": {
          "containsEpubBubbles": false,
          "containsImageBubbles": false
        },
        "imageLinks": {
          "smallThumbnail": "http://books.google.com/books/content?id=4Y18AQAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api",
          "thumbnail": "http://books.google.com/books/content?id=4Y18AQAAQBAJ&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api"
        },
        "language": "en",
        "previewLink": "http://books.google.co.jp/books?id=4Y18AQAAQBAJ&pg=PR4&dq=978-0-472-05206-6&hl=&cd=1&source=gbs_api",
        "infoLink": "http://books.google.co.jp/books?id=4Y18AQAAQBAJ&dq=978-0-472-05206-6&hl=&source=gbs_api",
        "canonicalVolumeLink": "https://books.google.com/books/about/Writing_History_in_the_Digital_Age.html?hl=&id=4Y18AQAAQBAJ"
      },
      "saleInfo": {
        "country": "JP",
        "saleability": "NOT_FOR_SALE",
        "isEbook": false
      },
      "accessInfo": {
        "country": "JP",
        "viewability": "PARTIAL",
        "embeddable": true,
        "publicDomain": false,
        "textToSpeechPermission": "ALLOWED",
        "epub": {
          "isAvailable": false
        },
        "pdf": {
          "isAvailable": false
        },
        "webReaderLink": "http://play.google.com/books/reader?id=4Y18AQAAQBAJ&hl=&printsec=frontcover&source=gbs_api",
        "accessViewStatus": "SAMPLE",
        "quoteSharingAllowed": false
      },
      "searchInfo": {
        "textSnippet": "(Digital humanities) ISBN 978-0-472-07206-4 (hardback) — ISBN \u003cb\u003e978-0-472-05206-6\u003c/b\u003e (paper) — ISBN 978-0-472-02991-4(e-book) 1. History—Methodology. 2. Academic writing—Data processing&nbsp;..."
      }
    }
  ]
}
```

# Google BooksのAPIから必要なデータを取得するための関数を定義


In [ ]:
def get_from_gbook(isbn, not_found_list):

    gbook_url = 'https://www.googleapis.com/books/v1/volumes?q=isbn:'
    # googlebooksでは、このuriの後ろにisbnを入れると書誌情報が出るようになっている（これがAPI）
    res = requests.get(gbook_url + isbn)
    # gbook_urlの後ろにisbnを足したurlにアクセスしてソースコードを取得し、requestsオブジェクトに格納している
    time.sleep(1)
    # 相手のサーバーに負荷をかけないように、1つのurlの作業が終わったら1秒処理を止める
    res_text = res.text
    # resに格納したソースコードからテキストを取り出している
    res_json = json.loads(res_text)
    # ソースコードはJSON形式なので、コンピュータにこれはJSON形式だと教えている
    # res_jsonは辞書形式

    if res_json['totalItems'] == 0:
      # res_jsonという辞書の'totalItems'というkeyに対応するvalueが0になっていたら（＝そのisbnがgooglebooks上に見つからなかった時の処理）
        not_found_list.append(isbn)
        # 関数の引数として指定したnot_found_listにisbnを追加する
        author = ''
        # author以下の情報は何も追加しない
        title = ''
        publisher = ''
        year = ''

    else:
      # そうでなければ（＝googlebooks上でisbnが見つかれば）以下の処理を行う
        authors_list = res_json['items'][0]['volumeInfo']['authors']
        # res_jsonの'items'というkeyに対応するvalueの0番目（最初の要素）の中の
        # 'volumeInfo'というkeyに対応するvalue（辞書になっている）の
        # 'authors'というkeyに対応するvalueをauthors_listに格納する
        author = ', '.join(authors_list)
        # authors_listに含まれる要素を特定の文字列（ここでは', '）で結合する
        title = res_json['items'][0]['volumeInfo']['title']
        try:
            publisher = res_json['items'][0]['volumeInfo']['publisher']
            # エラーが起きる可能性のあるコードをtryに入れる
        except KeyError:
            publisher = ''
            # エラーが起きたら、exceptの処理をしてもらう

        year = res_json['items'][0]['volumeInfo']['publishedDate']

    d = make_dict(author, title, publisher, year, isbn)
    d['情報源'] = 'Google Books'
    # ciniiの時と同じ処理。こちらはgoogle booksからの情報なので情報源がGoogle Booksになっている

    return d, not_found_list

# メインの処理開始

関数を定義し終わったので、具体的な書誌情報取得に移ります。

### csvファイルからisbnのリストを作成・整形する

In [ ]:
f = open('/content/drive/MyDrive/Colab Notebooks/isbn.csv', 'r', encoding='utf-8')
# open関数を使ってisbn.csvというCSVファイルを読み込みモード（'r'）で開いている
# encoding='utf-8'は、ファイルがUTF-8の文字コードでエンコードされていることを指定している
isbn_list = pandas.read_csv(f, header=None).values.tolist()
# pandasライブラリに含まれる.read_csv関数を使ってCSVファイルを読み込み、Pythonのリストに変換している
# header=Noneは、CSVファイルにヘッダー行がないことを意味する
isbn_list_new = []
# これからフォーマットされたisbnを読み込む処理を始めるので、空のリストを用意する
for l in isbn_list:
  # for文は繰り返し処理。isbn_listの中の要素を前から1つずつlという変数に格納して以下の処理をする
  isbn_list_new.append(str(l[0]).translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)})))
  # isbn_listに含まれる要素は全角文字のことがあり得るので、これを半角文字に変換する処理
  # lはリスト型なので、文字列を取り出すにはl[0]と指定する必要がある
  # str.maketransは変換テーブルを作成しており、全角の記号や数字を半角に変換するためのマッピングを定義している
  # .translateは、作成された変換テーブルに基づいて文字列の全角文字を半角に変換する


# 全角を半角に変えるコードは以下を参照
# https://qiita.com/YuukiMiyoshi/items/6ce77bf402a29a99f1bf

f.close()
# 最初に開いたファイルは必ず閉じる必要がある

In [ ]:
isbn_list

[[9784815810337],
 [9784560087077],
 [9784815807443],
 [9782130584223],
 [9784422211756],
 [9782130559092],
 [9784888882828]]

In [ ]:
type(isbn_list[0][0])

str

In [ ]:
isbn_list_new
# 作成されたisbn_list_newを出力してみよう

['9784815810337',
 '9784560087077',
 '9784815807443',
 '9782130584223',
 '9784422211756',
 '9782130559092',
 '9784888882828']

In [ ]:
type(isbn_list_new[0])

str

In [ ]:
# Colab上の数字の表示
text = "９７８７６７６７６" # 全角
text2 = "978767676" # 半角（英字入力）
text3 = "９７８７６７６７６" #半角(日本語入力)

### isbnのリストから書誌情報を取得する

In [ ]:
# cf.https://note.nkmk.me/python-csv-reader-writer/

ff = open('/content/drive/MyDrive/Colab Notebooks/20240110蔵書一覧表.tsv', 'w', encoding='utf-8')
## Colab Notebooksのフォルダ内に'20231220蔵書一覧表.tsv'というタイトルのファイルを新規作成（'w'は書き込みモード）する
writer = csv.DictWriter(ff, ["著者", "題名", "出版社", "出版年", "ISBN", "情報源"], delimiter="\t")
# csv.DictWriterは辞書形式のデータをCSVファイルに書き込むためのオブジェクト。各カラムのヘッダーを指定し、タブ区切り（\t）でデータを分割
writer.writeheader()
# writeheaderメソッドでヘッダー行をファイルに書き込む

not_found_list = []
# not_found_listという空のリストを作る（あるいは初期化）

for x in isbn_list_new:
    isbn = str(x) # isbnは文字型のはずだが、念のため文字型で指定しておく
    cinii_api = 'https://ci.nii.ac.jp/books/opensearch/search?isbn='
    res = requests.get(cinii_api + isbn)
    # requestsモジュールでソースコードを取得
    time.sleep(1)
    res_xml = bs4.BeautifulSoup(res.text, 'xml')
    # BeautifulSoupライブラリでxmlファイルを読み込めるようにする

    # print(res.text) -> ok

    if res_xml.totalResults.text != '0':
      # totalResultsが'0'でない場合CiNii Booksから情報を取得し、そうでない場合はGoogle Books APIから情報を取得する
      # totalResults=='0'はCiNii Books上にデータがないことを意味する
        d = get_from_cinii(isbn, res_xml)
        # get_from_ciniiという関数を呼び出し、実行する
    else:
        d, not_found_list = get_from_gbook(isbn, not_found_list)
        # get_from_gbookという関数を呼び出し、実行する
        # この関数はreturnでdとnot_found_listを返してくるので、左辺には2つの変数を指定しておく

    print(d)
    writer.writerow(d)
    # 各書籍の情報が辞書dに格納され、writer.writerow(d)でファイルに書き込まれる

ff.close()
# ファイルを閉じる
print(f'見つからなかった本のISBN: {not_found_list}')
# 見つからなかった書籍のisbnを出力する

{'著者': 'ロレイン・ダストン, ピーター・ギャリソン [著] ; 瀬戸口明久 [ほか] 訳', '題名': '客観性', '出版社': '名古屋大学出版会', '出版年': '2021', 'ISBN': '9784815810337', '情報源': 'cinii'}
{'著者': '目黒士門著', '題名': '現代フランス広文典', '出版社': '白水社', '出版年': '2015', 'ISBN': '9784560087077', '情報源': 'cinii'}
{'著者': 'I.ウォーラーステイン著 ; 川北稔訳', '題名': '重商主義と「ヨーロッパ世界経済」の凝集 : 1600-1750', '出版社': '名古屋大学出版会', '出版年': '2013', 'ISBN': '9784815807443', '情報源': 'cinii'}
{'著者': 'publié sous la direction de Lucien Bély', '題名': "Dictionnaire de l'Ancien Régime : royaume de France XVIe-XVIIIe siècle", '出版社': 'Quadrige : Presses universitaires de France', '出版年': '2010', 'ISBN': '9782130584223', '情報源': 'cinii'}
{'著者': 'ジュヌヴィエーヴ・ブレスク著 ; 遠藤ゆかり訳', '題名': 'ルーヴル美術館の歴史', '出版社': '創元社', '出版年': '2004', 'ISBN': '9784422211756', '情報源': 'cinii'}
{'著者': 'sous la direction de Michel Delon', '題名': 'Dictionnaire européen des lumières', '出版社': 'Quadrige/PUF', '出版年': '2007', 'ISBN': '9782130559092', '情報源': 'cinii'}
{'著者': '安成英樹著', '題名': 'フランス絶対王政とエリート官僚', '出版社': '日本エディタースクール出版部', '出版年': '1998'